In [57]:
from xml.etree import ElementTree as ET
import folium
import numpy as np
from shapely.geometry import Point, LineString
import pandas as pd
from sqlalchemy import create_engine
from math import radians, cos, sin, sqrt, atan2
import ast

Lógica para a execução do código
1. listar todos os carros que rodaram no dia
2. listar as linhas que estes carros rodaram
3. listar as rotas ociosas de cada linha
4. listar as rotas programadas de cada linha
5. listar a rota realizada pelo carro (dados mix)
6. fazer o comparativo entre realizada e programada
7. enviar email com os carros ofensores

Todos os dados estão no banco postgres.

In [58]:
dbname = 'poc-db'
user = 'adminpoc'
password = 'adminpoc'
host = 'localhost'
port = '5440' 

In [59]:
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')

In [60]:
ns = {'kml': 'http://www.opengis.net/kml/2.2'}

# Paths

In [61]:
planned_trip_path = '../notebooks/data/1.1 LINHAS RMTC [2023-11-29].kml'

# SQL

In [62]:
sql_geodata = """ 
select a.execution_date, a.registration, a.description, a.coordinates
    from hp.raw_geodata a
"""

In [63]:
sql_idle = """ 
select a.id, a.num_linha, a.descricao, b.direcao, b.pontos_geolocalizacao
from
    hp.raw_linhas a,
    hp.raw_trajetos b
where a.id = b.linha_id
"""

In [64]:
sql_schedule = """ 
SELECT id, "data", linha, carro
FROM hp.raw_escalas_programadas;
"""

In [65]:
sql_routes = """
SELECT id, linha_id, direcao, pontos_geolocalizacao
FROM hp.raw_trajetos;
"""

# Dataframes

In [66]:
df_trip_made = pd.read_sql(sql_geodata, engine)

In [67]:
df_trip_idle = pd.read_sql(sql_idle, engine)

In [68]:
df_schedule = pd.read_sql(sql_schedule, engine)

In [69]:
df_routes = pd.read_sql(sql_routes, engine)

# Functions

In [70]:
def read_and_extract_placemarks(file_path, namespace):
    """ 
    :param: file_path: caminho para o arquivo kml
    :return: lista de dicionários com os nomes e coordenadas dos placemarks

    A função lê um arquivo kml e extrai os placemarks, retornando uma lista de dicionários com os nomes e coordenadas dos placemarks.
    """
    tree = ET.parse(file_path)
    root = tree.getroot()
    placemarks = []
    for placemark in root.findall('.//kml:Placemark', namespace):
        name = placemark.find('kml:name', namespace).text if placemark.find('kml:name', namespace) is not None else "Unnamed"
        coordinates = placemark.find('.//kml:coordinates', namespace).text.strip() if placemark.find('.//kml:coordinates', namespace) is not None else "No coordinates"
        placemarks.append({'name': name, 'coordinates': coordinates})
    return placemarks

In [71]:
def parse_coordinates(coordinates_str):
    """
    param: coordinates_str: string de coordenadas (lat, long)
    return: lista de tuplas de coordenadas (long, lat)

    A função recebe uma string de coordenadas (lat, long) e retorna uma lista de tuplas de coordenadas (long, lat).
    """
    coords = coordinates_str.split()
    return [(float(coord.split(',')[1]), float(coord.split(',')[0])) for coord in coords]

In [72]:
def add_placemarks_to_map(placemarks, map):
    """ 
    param: placemarks: lista de dicionários com as informações dos placemarks
    return: map: objeto do tipo folium.Map

    Quando o arquivo conter somente um ponto, por exemplo uma tupla de coordenadas,ele será plotado como um marcador de viagem realizada. Quando houver
    uma lista de coordenadas, oriundas de um arquivo com trajetória, será plotado uma linha entre os pontos.
    """
    for i, placemark in enumerate(placemarks):
        parsed_coords = parse_coordinates(placemark['coordinates'])
        if len(parsed_coords) == 1:  # Para um único ponto
            folium.Marker(
                location=parsed_coords[0],
                icon=folium.Icon(icon='fa-van-shuttle', prefix='fa',color='blue')
            ).add_to(map)

        else:  # Para múltiplos pontos (trajetória)
            if 'VOLTA' in placemark['name']:
                folium.PolyLine(locations=parsed_coords, color='green', weight=2.5, opacity=1, dash_array='5, 5').add_to(map)

            else:  # Para outros casos, usa uma linha contínua
                folium.PolyLine(locations=parsed_coords, color='red', weight=2.5, opacity=1).add_to(map)

In [73]:
def haversine(lon1, lat1, lon2, lat2):
    # Raio da Terra em quilômetros
    R = 6371.0

    # Conversão de coordenadas de graus para radianos
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Diferenças nas coordenadas
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Fórmula de Haversine
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance

In [74]:
def extract_coords(coords_list):
    return [[coord['lat'], coord['lng']] for coord in coords_list]

# Transformações

## df_planned_trp

In [75]:
placemarks_planned = read_and_extract_placemarks(planned_trip_path, ns)

In [76]:
df_planned_trip = pd.DataFrame(placemarks_planned)

In [77]:
df_planned_trip['Código'] = df_planned_trip['name'].apply(lambda x: x.split('-', 1)[0])

In [78]:
df_planned_trip['identificacao'] = df_planned_trip['name'].apply(lambda x: x.split('-', 1)[1])
df_planned_trip['identificacao'] = df_planned_trip['identificacao'].apply(lambda x: x.split('- ', 1)[0])
df_planned_trip['identificacao'] = df_planned_trip['identificacao'].apply(lambda x: x.split(' ', 1)[0])

In [79]:
df_planned_trip['Código'] = df_planned_trip['name'].apply(lambda x: x.split('-', 1)[0])
df_planned_trip['Descrição e Sentido'] = df_planned_trip['name'].apply(lambda x: x.split(' - ', 1)[1] if ' - ' in x else '')
df_planned_trip['Descrição'] = df_planned_trip['Descrição e Sentido'].apply(lambda x: x.rsplit(' - ', 1)[0] if ' - ' in x else x)
df_planned_trip['Sentido'] = df_planned_trip['Descrição e Sentido'].apply(lambda x: x.rsplit(' - ', 1)[1] if ' - ' in x else '')
df_planned_trip.drop('Descrição e Sentido', axis=1, inplace=True)

In [80]:
de_para = {
    '1': 'A', '2': 'B', '3': 'C', '4': 'D', '5': 'E',
    '6': 'F', '7': 'G', '8': 'H', '9': 'I', '21': 'A',
    '22': 'B', '23': 'C', '24': 'D', '25': 'E', '26': 'F',
    '27': 'G', '28': 'H', '29': 'I', '31': 'A', '32': 'B',
    '33': 'C', '34': 'D', '35': 'E', '36': 'F', '37': 'G',
    '38': 'H', '39': 'I'
}

In [81]:
df_planned_trip['sublinha'] = df_planned_trip['identificacao'].apply(lambda x: de_para[x[:-1]])

In [82]:
df_planned_trip['concatenated'] = df_planned_trip['Código'].astype(str) + df_planned_trip['sublinha']

# Execution

* Lógica para a execução do código
* listar todos os carros que rodaram no dia
* listar as linhas que estes carros rodaram
* listar as rotas ociosas de cada linha
* listar as rotas programadas de cada linha
* listar a rota realizada pelo carro (dados mix)
* fazer o comparativo entre realizada e programada
* enviar email com os carros ofensores
* Todos os dados estão no banco postgres.

In [ ]:
#filtar data 2024-02-20 no dataframe df_schedule coluna data


In [88]:
df_schedule[df_schedule['carro'] == 20998]

,id,data,linha,carro
6597,6546,2024-04-15,026A,20998
6598,6551,2024-04-15,026A,20998
6599,6558,2024-04-15,026A,20998
6600,6562,2024-04-15,026B,20998
6601,6568,2024-04-15,026B,20998
6602,6574,2024-04-15,026B,20998
6603,6580,2024-04-15,026B,20998
6604,6588,2024-04-15,026B,20998
6605,6596,2024-04-15,026B,20998
6606,6597,2024-04-15,026A,20998


In [84]:
df_schedule

,id,data,linha,carro
0,1,2024-02-25,026A,20017
1,2,2024-02-25,275A,20017
2,3,2024-02-25,022B,20017
3,4,2024-02-25,403A,20017
4,5,2024-02-25,574A,20017
...,...,...,...,...
90658,90659,2024-04-12,025C,20805
90659,90660,2024-04-12,025C,20805
90660,90661,2024-04-12,025A,20805
90661,90662,2024-04-12,025A,20805
